In [1]:
'''
This code takes 4 variables from the HRRR data and saves them in separate files. 
The code only takes out the variables from surface to 500 hPa levels. 
Change them (variables and pressure levels) accordingly. 
'''

'\nThis code takes 4 variables from the HRRR data and saves them in separate files. \nThe code only takes out the variables from surface to 500 hPa levels. \n'

In [2]:
import os
import pygrib
import numpy as np
from netCDF4 import Dataset

# Parent folder path
parent_folder = r'D:\2Courses\8902\data\hrrr\20231026\hrrr'

# Variables to extract
variables_to_extract = ['Temperature', 'Dew point temperature', 'Relative humidity', 'U component of wind', 'V component of wind']

# Pressure level to filter
pressure_level = 500

# Function to process HRRR data
def process_hrrr_files():
    for day_folder in os.listdir(parent_folder):
        day_path = os.path.join(parent_folder, day_folder)
        if os.path.isdir(day_path):  # Check if it's a folder
            for time in ['00Z', '06Z', '12Z', '18Z']:
                grib_filename = f'hrrr.t{time.lower()}.wrfprsf00.grib2'
                grib_filepath = os.path.join(day_path, grib_filename)
                
                if os.path.exists(grib_filepath):
                    grbs = pygrib.open(grib_filepath)
                    data_dict = {}
                    lats, lons = None, None  # Initialize lat/lon variables
                    
                    for var in variables_to_extract:
                        try:
                            grb = grbs.select(name=var, level=pressure_level)[0]
                            data = grb.values
                            if lats is None or lons is None:
                                lats, lons = grb.latlons()  # Extract lat/lon once
                            data_dict[var.replace(' ', '_')] = data
                        except Exception as e:
                            print(f"Variable {var} not found or could not be processed in {grib_filepath}: {e}")
                    
                    # Save netCDF file with valid spatial dimensions
                    if lats is not None and lons is not None:
                        save_to_netcdf(data_dict, lats, lons, day_folder, time)

# Function to save data to netCDF
def save_to_netcdf(data_dict, lats, lons, day_folder, time):
    nc_filename = f"{day_folder}_{time}.nc"
    nc_filepath = os.path.join(parent_folder, day_folder, nc_filename)
    
    try:
        with Dataset(nc_filepath, 'w', format='NETCDF4') as nc_file:
            # Create dimensions
            nc_file.createDimension('latitude', lats.shape[0])
            nc_file.createDimension('longitude', lons.shape[1])
            
            # Create variables for latitude and longitude
            latitudes = nc_file.createVariable('latitude', 'f4', ('latitude', 'longitude'))
            longitudes = nc_file.createVariable('longitude', 'f4', ('latitude', 'longitude'))
            
            latitudes[:, :] = lats
            longitudes[:, :] = lons
            
            # Create variables for each data variable
            for var_name, var_data in data_dict.items():
                var = nc_file.createVariable(var_name, 'f4', ('latitude', 'longitude'), fill_value=np.nan)  # Handle missing values
                var[:, :] = var_data

        print(f"Saved netCDF file: {nc_filepath}")
    except Exception as e:
        print(f"Error saving netCDF file {nc_filepath}: {e}")

# Execute the function
process_hrrr_files()


Saved netCDF file: D:\2Courses\8902\data\hrrr\20231026\hrrr\20231001\20231001_00Z.nc
Saved netCDF file: D:\2Courses\8902\data\hrrr\20231026\hrrr\20231001\20231001_06Z.nc
Saved netCDF file: D:\2Courses\8902\data\hrrr\20231026\hrrr\20231001\20231001_12Z.nc
Saved netCDF file: D:\2Courses\8902\data\hrrr\20231026\hrrr\20231001\20231001_18Z.nc
Saved netCDF file: D:\2Courses\8902\data\hrrr\20231026\hrrr\20231002\20231002_00Z.nc
Saved netCDF file: D:\2Courses\8902\data\hrrr\20231026\hrrr\20231002\20231002_06Z.nc
Saved netCDF file: D:\2Courses\8902\data\hrrr\20231026\hrrr\20231002\20231002_12Z.nc
Saved netCDF file: D:\2Courses\8902\data\hrrr\20231026\hrrr\20231002\20231002_18Z.nc
Saved netCDF file: D:\2Courses\8902\data\hrrr\20231026\hrrr\20231003\20231003_00Z.nc
Saved netCDF file: D:\2Courses\8902\data\hrrr\20231026\hrrr\20231003\20231003_06Z.nc
Saved netCDF file: D:\2Courses\8902\data\hrrr\20231026\hrrr\20231003\20231003_12Z.nc
Saved netCDF file: D:\2Courses\8902\data\hrrr\20231026\hrrr\20231


KeyboardInterrupt

